In [9]:
import pandas as pd

file_path = r"C:\Users\HR\Desktop\Workspace\데이터톤\마케팅\merged_data.csv"
df = pd.read_csv(file_path)

print(df.head())

                  event_time event_type  product_id          category_id  \
0  2019-10-01 00:00:00+00:00       cart     5773203  1487580005134238553   
1  2019-10-01 00:00:03+00:00       cart     5773353  1487580005134238553   
2  2019-10-01 00:00:07+00:00       cart     5881589  2151191071051219817   
3  2019-10-01 00:00:07+00:00       cart     5723490  1487580005134238553   
4  2019-10-01 00:00:15+00:00       cart     5881449  1487580013522845895   

    brand  price    user_id                          user_session  \
0  runail   2.62  463240011  26dd6e6e-4dac-4778-8d2c-92e149dab885   
1  runail   2.62  463240011  26dd6e6e-4dac-4778-8d2c-92e149dab885   
2  lovely  13.48  429681830  49e8d843-adf3-428b-a2c3-fe8bc6a307c9   
3  runail   2.62  463240011  26dd6e6e-4dac-4778-8d2c-92e149dab885   
4  lovely   0.56  429681830  49e8d843-adf3-428b-a2c3-fe8bc6a307c9   

           event_time_moscow  event_month  \
0  2019-10-01 03:00:00+03:00         10.0   
1  2019-10-01 03:00:03+03:00         1

In [11]:
import pandas as pd

# ✅ 1. 원본 데이터에서 필요한 컬럼만
events_clean = df[['event_time', 'event_type', 'product_id', 'brand', 'price', 'user_id', 'user_session', 'event_month']]
events_clean.to_csv('events_clean.csv', index=False)

print("✅ [1단계] events_clean.csv 저장 완료")
print("행 개수:", len(events_clean))
print("컬럼:", list(events_clean.columns))
print(events_clean.head(5))
print("-"*80)

# ✅ 2. 사용자별 이벤트 수
user_event_counts = df.groupby(['user_id', 'event_type']).size().unstack(fill_value=0)

# ✅ 구매금액 합계
user_purchase_amount = df[df['event_type'] == 'purchase'].groupby('user_id')['price'].sum()

# ✅ 세션 수
user_sessions = df.groupby('user_id')['user_session'].nunique()

# ✅ 합치기
user_summary = user_event_counts.copy()
user_summary['purchase_amount'] = user_purchase_amount
user_summary['session_count'] = user_sessions
user_summary = user_summary.fillna(0).reset_index()

# ✅ 전환율 추가
user_summary['conversion_rate'] = user_summary.apply(
    lambda x: x['purchase'] / x['view'] if x['view'] > 0 else 0, axis=1
)

# ✅ 저장
user_summary.to_csv('user_summary.csv', index=False)

print("✅ [2단계] user_summary.csv 저장 완료")
print("총 사용자 수:", len(user_summary))
print(user_summary.head(10))
print("-"*80)

# ✅ 3. 브랜드 다양성 추가
brand_diversity = df[df['event_type'] == 'purchase'].groupby('user_id')['brand'].nunique()
user_summary['brand_count'] = user_summary['user_id'].map(brand_diversity).fillna(0)

# ✅ 저장
user_summary.to_csv('user_summary_with_brand.csv', index=False)

print("✅ [3단계] user_summary_with_brand.csv 저장 완료")
print(user_summary[['user_id','purchase_amount','brand_count','conversion_rate']].head(10))
print("-"*80)

# ✅ 4. Top 10 유저
top_users = user_summary.sort_values('purchase_amount', ascending=False).head(10)
top_users.to_csv('top_users.csv', index=False)

print("✅ [4단계] Top 10 유저 저장 완료")
print(top_users[['user_id','purchase_amount','view','cart','purchase','conversion_rate']])
print("-"*80)

print("🎯 저장된 파일 목록:")
print("1) events_clean.csv")
print("2) user_summary.csv")
print("3) user_summary_with_brand.csv")
print("4) top_users.csv")



✅ [1단계] events_clean.csv 저장 완료
행 개수: 20688111
컬럼: ['event_time', 'event_type', 'product_id', 'brand', 'price', 'user_id', 'user_session', 'event_month']
                  event_time event_type  product_id   brand  price    user_id  \
0  2019-10-01 00:00:00+00:00       cart     5773203  runail   2.62  463240011   
1  2019-10-01 00:00:03+00:00       cart     5773353  runail   2.62  463240011   
2  2019-10-01 00:00:07+00:00       cart     5881589  lovely  13.48  429681830   
3  2019-10-01 00:00:07+00:00       cart     5723490  runail   2.62  463240011   
4  2019-10-01 00:00:15+00:00       cart     5881449  lovely   0.56  429681830   

                           user_session  event_month  
0  26dd6e6e-4dac-4778-8d2c-92e149dab885         10.0  
1  26dd6e6e-4dac-4778-8d2c-92e149dab885         10.0  
2  49e8d843-adf3-428b-a2c3-fe8bc6a307c9         10.0  
3  26dd6e6e-4dac-4778-8d2c-92e149dab885         10.0  
4  49e8d843-adf3-428b-a2c3-fe8bc6a307c9         10.0  
------------------------------

In [12]:
# ✅ 구매금액 기준으로 내림차순 정렬 후 순위 매기기
user_summary['rank'] = user_summary['purchase_amount'].rank(method='dense', ascending=False).astype(int)

# ✅ 정렬
user_summary_ranked = user_summary.sort_values(['rank', 'purchase_amount'], ascending=[True, False])

# ✅ 저장
user_summary_ranked.to_csv('user_summary_ranked.csv', index=False)

print("✅ [랭킹 포함] user_summary_ranked.csv 저장 완료")
print(user_summary_ranked.head(20))  # 상위 20명 확인


✅ [랭킹 포함] user_summary_ranked.csv 저장 완료
event_type    user_id  cart  purchase  remove_from_cart  view  \
1669        150318419   592       370               132  1029   
298568      531900924  1006       372               532  3678   
599117      562167663   560       226                13    34   
153949      471197852   832       389               318  2036   
150308      469299888  1707       603               703   357   
8740        247216055  1007       409               498  1207   
501901      557790271   844       438               381   283   
659975      565036131  1366       255              1637   408   
94679       431950134   384       400                66    16   
801356      572081598   599       273               174   993   
50953       381816972   797       500               376    57   
1488364     612372707   243       131                73    57   
675287      565745379   300       124               108  1036   
1195168     596971707   399       255             

In [13]:
# ✅ 1. 필요한 컬럼만 선택
funnel_df = df[['brand', 'event_month', 'event_type', 'user_id']].copy()

# ✅ 2. 이벤트 타입 필터 (view/cart/purchase만)
funnel_df = funnel_df[funnel_df['event_type'].isin(['view', 'cart', 'purchase'])]

# ✅ 3. 브랜드, 월, 이벤트 타입별 카운트
funnel_counts = funnel_df.groupby(['brand', 'event_month', 'event_type']).size().unstack(fill_value=0).reset_index()

# ✅ 4. 전환율 계산
# View → Cart, Cart → Purchase, View → Purchase
funnel_counts['view_to_cart_rate'] = funnel_counts.apply(
    lambda x: x['cart'] / x['view'] if x['view'] > 0 else 0, axis=1
)
funnel_counts['cart_to_purchase_rate'] = funnel_counts.apply(
    lambda x: x['purchase'] / x['cart'] if x['cart'] > 0 else 0, axis=1
)
funnel_counts['overall_conversion_rate'] = funnel_counts.apply(
    lambda x: x['purchase'] / x['view'] if x['view'] > 0 else 0, axis=1
)

# ✅ 5. 저장
funnel_counts.to_csv('brand_month_funnel.csv', index=False)

print("✅ [브랜드별+월별 퍼널] brand_month_funnel.csv 저장 완료")
print(funnel_counts.head(10))


✅ [브랜드별+월별 퍼널] brand_month_funnel.csv 저장 완료
event_type     brand  event_month  cart  purchase  view  view_to_cart_rate  \
0           airnails          1.0  4730      1288  4884           0.968468   
1           airnails          2.0  4641      1160  4456           1.041517   
2           airnails         10.0  6998      1741  6170           1.134198   
3           airnails         11.0  6596      1928  6497           1.015238   
4           airnails         12.0  4732      1383  5334           0.887139   
5              almea          1.0   147        11   683           0.215227   
6              almea          2.0   128        36   751           0.170439   
7              almea         10.0   300        34   741           0.404858   
8              almea         11.0   178        34   775           0.229677   
9              almea         12.0   115        31   603           0.190713   

event_type  cart_to_purchase_rate  overall_conversion_rate  
0                        0.272304   

In [14]:
# ✅ 1. 이벤트 기반 퍼널 (이미 작성한 것)
funnel_df = df[['brand', 'event_month', 'event_type', 'user_id']].copy()
funnel_df = funnel_df[funnel_df['event_type'].isin(['view', 'cart', 'purchase'])]

# 이벤트 개수 기반 퍼널
funnel_counts_events = funnel_df.groupby(['brand', 'event_month', 'event_type']).size().unstack(fill_value=0).reset_index()

# 전환율 계산
funnel_counts_events['view_to_cart_rate'] = funnel_counts_events.apply(
    lambda x: x['cart'] / x['view'] if x['view'] > 0 else 0, axis=1
)
funnel_counts_events['cart_to_purchase_rate'] = funnel_counts_events.apply(
    lambda x: x['purchase'] / x['cart'] if x['cart'] > 0 else 0, axis=1
)
funnel_counts_events['overall_conversion_rate'] = funnel_counts_events.apply(
    lambda x: x['purchase'] / x['view'] if x['view'] > 0 else 0, axis=1
)

# 저장
funnel_counts_events.to_csv('brand_month_funnel_events.csv', index=False)
print("✅ brand_month_funnel_events.csv 저장 완료")


# ✅ 2. 유저 기반 퍼널
# 유저별 중복 제거 후 count
funnel_unique = funnel_df.drop_duplicates(subset=['user_id', 'brand', 'event_month', 'event_type'])
funnel_counts_users = funnel_unique.groupby(['brand', 'event_month', 'event_type']).size().unstack(fill_value=0).reset_index()

# 전환율 계산 (유저 기준)
funnel_counts_users['view_to_cart_rate'] = funnel_counts_users.apply(
    lambda x: x['cart'] / x['view'] if x['view'] > 0 else 0, axis=1
)
funnel_counts_users['cart_to_purchase_rate'] = funnel_counts_users.apply(
    lambda x: x['purchase'] / x['cart'] if x['cart'] > 0 else 0, axis=1
)
funnel_counts_users['overall_conversion_rate'] = funnel_counts_users.apply(
    lambda x: x['purchase'] / x['view'] if x['view'] > 0 else 0, axis=1
)

# 저장
funnel_counts_users.to_csv('brand_month_funnel_users.csv', index=False)
print("✅ brand_month_funnel_users.csv 저장 완료")

# 미리보기 출력
print("\n[이벤트 기반 퍼널 상위 5행]")
print(funnel_counts_events.head())
print("\n[유저 기반 퍼널 상위 5행]")
print(funnel_counts_users.head())


✅ brand_month_funnel_events.csv 저장 완료
✅ brand_month_funnel_users.csv 저장 완료

[이벤트 기반 퍼널 상위 5행]
event_type     brand  event_month  cart  purchase  view  view_to_cart_rate  \
0           airnails          1.0  4730      1288  4884           0.968468   
1           airnails          2.0  4641      1160  4456           1.041517   
2           airnails         10.0  6998      1741  6170           1.134198   
3           airnails         11.0  6596      1928  6497           1.015238   
4           airnails         12.0  4732      1383  5334           0.887139   

event_type  cart_to_purchase_rate  overall_conversion_rate  
0                        0.272304                 0.263718  
1                        0.249946                 0.260323  
2                        0.248785                 0.282172  
3                        0.292298                 0.296752  
4                        0.292265                 0.259280  

[유저 기반 퍼널 상위 5행]
event_type     brand  event_month  cart  purchase  vi

# 주요 인사이트 
    ✅ 1. 이벤트 기반 vs 유저 기반 차이
    이벤트 기반(view_to_cart_rate) → 1.0 이상인 경우 있음
    → 한 명의 유저가 카트 이벤트를 반복한 경우 (ex: airnails 브랜드 1월 = 96.8%)

    유저 기반(view_to_cart_rate) → 항상 ≤ 1
    → 중복 제거 후 비율, 실제 유저 행동을 더 잘 반영
    ✅ 결론: 이벤트 기반은 행동 강도, 유저 기반은 전환 가능성 확인용

    ✅ 2. 전환율 비교
    airnails(11월):

    이벤트 기반 Cart→Purchase = 29.2%

    유저 기반 Cart→Purchase = 36.6%
    → 유저당 비율이 더 높은 이유: 장바구니 반복 이벤트로 이벤트 기반 비율이 희석됨
    ✅ 결론: 퍼널 최적화는 유저 기준 데이터가 더 정확한 인사이트 제공

    ✅ 3. 월별 변화
    1월 vs 12월 비교:

    이벤트 기반 View→Cart 비율 하락 (0.96 → 0.88)

    유저 기반 View→Cart 비율도 하락 (0.40 → 0.38)
    ✅ 해석: 연말에 신규 유저는 많지만 카트까지 진행하는 비율이 낮음 → 프로모션 부족 or 충성도 낮은 유저 유입

    ✅ 4. 브랜드 전략 포인트
    특정 브랜드에서 View→Cart는 높은데 Cart→Purchase가 낮음
    → 장바구니 방치 → 리마케팅 타겟 후보

    반대로 View→Cart는 낮지만 Cart→Purchase가 높음
    → 구매 의도 강한 고객층 → 핵심 타겟 집중 전략



In [17]:
# ✅ 세션별 이벤트 카운트
session_event_counts = df.groupby(['user_key', 'event_type']).size().unstack(fill_value=0)

# ✅ 세션별 구매금액 합계
session_purchase_amount = df[df['event_type'] == 'purchase'].groupby('user_key')['price'].sum()

# ✅ 합치기
session_summary = session_event_counts.copy()
session_summary['purchase_amount'] = session_purchase_amount
session_summary = session_summary.fillna(0).reset_index()

print("✅ 세션별 요약")
print(session_summary.head())


✅ 세션별 요약
event_type                                       user_key  cart  purchase  \
0           10001501049e717f6-d1c5-4a87-88ec-20c2cc9c3637     0         0   
1           100015010852f0890-41c4-4870-92c3-3d13e7ea22bc     0         0   
2           100015010c39a7ce1-cea7-4da8-bf8e-13d1d26ace6f     0         0   
3           100032619532d31f6-8f96-48cd-8e0b-c2024c2adc8e     2         0   
4           1000516324d0aebb5-6d4e-408b-89fc-0cfd2dd645c1     0         0   

event_type  remove_from_cart  view  purchase_amount  
0                          0     1              0.0  
1                          0     1              0.0  
2                          0     2              0.0  
3                          0     0              0.0  
4                          0     3              0.0  


In [20]:
session_summary['conversion_rate'] = session_summary.apply(
    lambda x: x['purchase'] / x['view'] if x['view'] > 0 else 0, axis=1
)
session_summary.to_csv('session_summary.csv', index=False)
print("✅ session_summary.csv 저장 완료")


✅ session_summary.csv 저장 완료


# ✅ 세션별 요약
    ✅ 1. 세션의 대부분이 단일 이벤트
    → 세션 대부분이 1~2개의 View 이벤트 후 종료
    ✅ 결론: 깊은 탐색 없이 바로 이탈하는 세션이 많음
    → 이탈 원인 분석 필요 (UX 개선, 추천상품 강화 등)

    ✅ 2. 장바구니 추가(cart)만 있고 view=0인 세션 존재
       → 이벤트 로깅 순서 문제 or 뷰 없이 카트 진입(재방문)
    ✅ 결론: 세션 로깅 품질 검토 필요,
    또는 장바구니 바로 접근 기능이 활발히 사용되는 UX 가능성

    ✅ 3. 구매 세션 희소
    상위 5행 모두 purchase=0, purchase_amount=0.0

    전체 데이터에서도 구매 세션 비율 낮을 것으로 예상
    ✅ 결론: 구매 전환율이 낮음,
    → 퍼널 분석에서 본 것처럼 대부분 세션은 탐색(View) 단계에서 이탈

    ✅ 4. remove_from_cart 비율도 낮음
    remove_from_cart=0 대부분 →
    ✅ 해석: 장바구니 관리 기능을 거의 사용하지 않음
    → 장바구니 유지 시간이 길 가능성 → 장바구니 방치율 분석 필요



In [22]:
# ✅ 세션별 이벤트 카운트
session_event_counts = df.groupby(['user_key', 'event_type']).size().unstack(fill_value=0)

# ✅ 세션별 구매 금액
session_purchase_amount = df[df['event_type'] == 'purchase'].groupby('user_key')['price'].sum()

# ✅ 합치기
session_summary = session_event_counts.copy()
session_summary['purchase_amount'] = session_purchase_amount
session_summary = session_summary.fillna(0).reset_index()

# ✅ 세션별 퍼널 플래그
session_summary['has_view'] = session_summary['view'] > 0
session_summary['has_cart'] = session_summary['cart'] > 0
session_summary['has_purchase'] = session_summary['purchase'] > 0

# ✅ CSV 저장
session_summary.to_csv('session_summary.csv', index=False)
print("✅ session_summary.csv 저장 완료")
print(session_summary.head())


✅ session_summary.csv 저장 완료
event_type                                       user_key  cart  purchase  \
0           10001501049e717f6-d1c5-4a87-88ec-20c2cc9c3637     0         0   
1           100015010852f0890-41c4-4870-92c3-3d13e7ea22bc     0         0   
2           100015010c39a7ce1-cea7-4da8-bf8e-13d1d26ace6f     0         0   
3           100032619532d31f6-8f96-48cd-8e0b-c2024c2adc8e     2         0   
4           1000516324d0aebb5-6d4e-408b-89fc-0cfd2dd645c1     0         0   

event_type  remove_from_cart  view  purchase_amount  has_view  has_cart  \
0                          0     1              0.0      True     False   
1                          0     1              0.0      True     False   
2                          0     2              0.0      True     False   
3                          0     0              0.0     False      True   
4                          0     3              0.0      True     False   

event_type  has_purchase  
0                  False  
1   

In [24]:
# ✅ 세션별 이벤트 경로 추출
session_paths = df.groupby('user_key')['event_type'].apply(lambda x: ' > '.join(x)).reset_index()

# ✅ 경로별 빈도수 집계
path_counts = session_paths['event_type'].value_counts().reset_index()
path_counts.columns = ['path', 'count']

# ✅ 순위 추가
path_counts['rank'] = path_counts['count'].rank(method='dense', ascending=False).astype(int)

# ✅ 정렬
path_counts = path_counts.sort_values(['count', 'path'], ascending=[False, True])

# ✅ 저장
path_counts.to_csv('session_paths_ranked.csv', index=False)

# ✅ 출력 예시 (상위 20개)
print("✅ 전체 세션 경로 순위 저장 완료 (session_paths_ranked.csv)")
print(path_counts.head(20))


✅ 전체 세션 경로 순위 저장 완료 (session_paths_ranked.csv)
                                                 path    count  rank
0                                                view  2819857     1
1                                         view > view   318792     2
2                                  view > view > view   125080     3
3                                         view > cart   110011     4
4                                                cart    57215     5
5                           view > view > view > view    56837     6
6                           view > cart > cart > cart    55118     7
7                    view > view > view > view > view    34739     8
8                                         cart > cart    24311     9
9             view > view > view > view > view > view    21146    10
10                                 view > cart > cart    15217    11
11                                 cart > cart > cart    14080    12
12                                 view > cart > view   

# 핵심 인사이트
    ✅ 1. 단일 행동 세션이 절대 다수
    1위: view → 2,819,857회 (압도적)

    4위: view > cart → 110,011회

    5위: cart → 57,215회
    해석:

    대부분의 세션은 조회(view)만 하고 종료

    일부는 카트까지 갔다가 이탈

    구매 이벤트가 포함된 패턴은 Top 20 안에 없음
    → 전환율이 매우 낮다는 것을 재확인

    ✅ 2. 반복 이벤트 패턴 강함
    view > view, view > view > view 등 뷰 반복 → Top 2, 3, 6, 8

    cart > cart, cart > cart > cart → Top 9, 12, 20
    해석:
    
    동일 행동 반복 비율 높음 → 탐색 집중형 고객 많음

    카트 반복: 아마도 여러 상품 장바구니 담기 or 장바구니 수정
    → 장바구니 UI 개선 또는 추천 상품 최적화 필요

    ✅ 3. 구매 이벤트가 없는 상위 패턴
    Top 20 안에 purchase가 없음 → 구매는 매우 드물거나 긴 경로 후 발생
    해석:

    구매 전환은 극소수 → UX 개선 또는 결제 프로세스 리마케팅 중요

    ✅ 4. remove_from_cart 패턴 등장
    remove_from_cart 단독 9,050회, 반복 패턴도 있음
    해석:

    고객이 장바구니 관리 행위를 하지만 최종 구매로 이어지지 않는 경우 많음
    → 가격 민감도 or 재고/옵션 문제 가능
    → 가격 인센티브, 추천 대체상품 제공 전략 필요

    ✅ 전략 포인트
    이탈 방지 전략:

    View → Cart 비율 낮음 → 카트 유도 팝업, 추천상품 강화

    장바구니 방치 해결:

    Cart 반복 & Remove 이벤트 많음 → 쿠폰 제공, 심리적 가격 전략

    전환율 향상:

    Top 패턴에서 Purchase가 없으므로, 결제 UX 검토 + 푸시알림 전략 필요



In [25]:
# ✅ 세션 시작 & 종료 시간
df['event_time'] = pd.to_datetime(df['event_time'])
session_time = df.groupby('user_key')['event_time'].agg(['min', 'max'])
session_time['session_duration_sec'] = (session_time['max'] - session_time['min']).dt.total_seconds()

# 구매 여부 추가
session_time['has_purchase'] = session_time.index.isin(session_summary[session_summary['has_purchase']].user_key)

session_time.to_csv('session_duration.csv', index=True)
print(session_time.head())


                                                                    min  \
user_key                                                                  
10001501049e717f6-d1c5-4a87-88ec-20c2cc9c3637 2019-10-28 11:44:26+00:00   
100015010852f0890-41c4-4870-92c3-3d13e7ea22bc 2019-10-10 14:11:28+00:00   
100015010c39a7ce1-cea7-4da8-bf8e-13d1d26ace6f 2019-10-28 11:40:48+00:00   
100032619532d31f6-8f96-48cd-8e0b-c2024c2adc8e 2019-11-22 11:44:46+00:00   
1000516324d0aebb5-6d4e-408b-89fc-0cfd2dd645c1 2020-02-23 14:41:35+00:00   

                                                                    max  \
user_key                                                                  
10001501049e717f6-d1c5-4a87-88ec-20c2cc9c3637 2019-10-28 11:44:26+00:00   
100015010852f0890-41c4-4870-92c3-3d13e7ea22bc 2019-10-10 14:11:28+00:00   
100015010c39a7ce1-cea7-4da8-bf8e-13d1d26ace6f 2019-10-28 11:42:47+00:00   
100032619532d31f6-8f96-48cd-8e0b-c2024c2adc8e 2019-11-22 11:49:03+00:00   
1000516324d0aebb5-6d4e-4